# Transfer Learning - Universal Sentence Encoder

## The stupid proxy..

In [1]:
import os
os.environ['http_proxy'] = "http://proxy.mms-dresden.de:8080"
os.environ['https_proxy'] = "http://proxy.mms-dresden.de:8080"

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\dnho\AppData\Local\conda\conda\envs\tfdeeplearning2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Loading the model 

In [3]:
model_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 

In [4]:
import hashlib
# The path where tf-hub will cache the model (use an absolute path..) 
os.environ["TFHUB_CACHE_DIR"] = 'C:/Users/dnho/Desktop/universal_sentence_encoder/model'

#TF-hub will store the name as hex
hashlib.sha1(model_url.encode("utf8")).hexdigest()

'1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47'

In [20]:
# Reduce logging output.
#tf.logging.set_verbosity(tf.logging.ERROR)

In [21]:
%%time
# Initial download takes a while till the model is downloaded from tf-hub (~1GB)
model = hub.Module(model_url, trainable = True) # trainable = True for Transfer Learning!!

Wall time: 1.74 s


## Loading the Text into a Dataframe

In [22]:
data_dir = "dataset_1"

In [23]:
data = {}
data["sentence"] = []
data["class"] = []

for root, dirs, files in os.walk(data_dir):
    for _dir in dirs: 
        for txt_file in [x for x in os.listdir(os.path.join(root, _dir)) if x.endswith((".txt", ".TXT"))]:
            class_name = _dir
            file_name = os.path.join(root, _dir, txt_file)
            file = open(file_name, "r")
            txt = file.read()
            file.close()
            data["sentence"].append(txt)
            data["class"].append(class_name)
            
df = pd.DataFrame.from_dict(data)
del data

In [26]:
messages = df["sentence"]

In [28]:
messages

0                                           pwnrpnbpem
1    When Dr. Richard J. Gatling designed his gun, ...
2     Thi is a stupid test. Doesnt make sense at all. 
3    Machine learning is a method of data analysis ...
Name: sentence, dtype: object

In [ ]:
embedded_text_feature_column = hub.text_embedding_column(
    key="sentence", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")

In [27]:
%%time
with tf.Session() as session: 
    # Initializing global variables in the graph 
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    
    message_embeddings = session.run(model(messages))
    print("Return result as np.ndarray: ")
    print(message_embeddings)
    
    print("\n Pretty print: ")
    
    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join((str(x) for x in message_embedding[:3]))
        print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

TypeError: Can't convert 'text': data type not understood